In [ ]:
# pitch generation around a specific pitch profile

import numpy as np
import matplotlib.pyplot as plt

data_path = "pitch_profiles\pitch_g_42_49.txt"

# Load the data from the text file
data = np.loadtxt(data_path)

# make a 100 copies of the data with slight variations to only the second line
data_copies = []
for i in range(100):
    # Create a copy of the data
    data_copy = data.copy()
    
    # Modify the second line with a small random continuous variation
    add_min = np.random.choice([0, 1])
    if add_min == 1:
        data_copy[1] += np.random.uniform(0.00005, 0.00015)
    variation_type = np.random.choice(['gaussian', 'exponential', 'bump'])
    if variation_type == 'gaussian':
        data_copy[1] += np.random.normal(0, 0.00001, size=data_copy[1].shape)
    elif variation_type == 'exponential':
        data_copy[1] += np.random.exponential(0.000007, size=data_copy[1].shape)
    elif variation_type == 'bump':
        amplitude = np.random.uniform(-0.00001, 0.00015)
        mean = np.random.uniform(-0.01, 0.01)
        std_dev = np.random.uniform(0.001, 0.005)
        data_copy[1] += amplitude * np.exp(-0.5 * ((data_copy[0] - mean) / std_dev) ** 2)
    
    # Append the modified copy to the list
    data_copies.append(data_copy)

    # only take the six number after the comma
    data_copy[1] = np.round(data_copy[1], 6)

print(data_copies[0])


plt.figure()
plt.plot(data[0], data[1], label='Original Data', color='blue')
plt.plot(data_copies[0][0], data_copies[0][1], label='Modified Data', color='red')
plt.plot(data_copies[1][0], data_copies[1][1], label='Modified Data', color='green')
plt.plot(data_copies[2][0], data_copies[2][1], label='Modified Data', color='orange')
plt.plot(data_copies[3][0], data_copies[3][1], label='Modified Data', color='purple')
plt.xlabel('X-axis Label')
plt.ylabel('Y-axis Label')
plt.title('Data Comparison')
plt.legend()
plt.grid()
plt.show()

# save all the modified data to a new folder
folder_path = "pitch_profiles/modified_data"
import os
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

for i, data_copy in enumerate(data_copies):
    # Save each modified copy to a separate file
    file_path = os.path.join(folder_path, f"coil_h_profile_42{i}.txt")
    np.savetxt(file_path, data_copy)
    print(f"Saved modified data to {file_path}")

In [ ]:
# pitch generation for profile families

import numpy as np
import matplotlib.pyplot as plt

def generate_profile(profile_func, profile_X, min_y, max_y, derivative_max, step, retry_limit=50):
    """
    Generalized function to generate a profile with constraints.

    Args:
        profile_func (function): Function to generate the profile.
        profile_X (np.ndarray): X-axis values for the profile.
        min_y (float): Minimum Y value for the profile.
        max_y (float): Maximum Y value for the profile.
        derivative_max (float): Maximum allowed derivative.
        step (float): Step size for derivative calculation.
        retry_limit (int): Maximum number of retries.

    Returns:
        np.ndarray: Generated profile or None if constraints are not met.
    """
    for _ in range(retry_limit):
        profile = profile_func(profile_X, min_y, max_y)
        in_bounds = np.max(profile) < max_y and np.min(profile) > min_y
        derivative_ok = np.max(np.abs(np.diff(profile))) < derivative_max * step
        if in_bounds and derivative_ok:
            return profile
    return None

# Example profile generation functions
def linear_profile_func(profile_X, min_y, max_y):
    b = np.random.uniform(min_y, max_y)
    min_a = (max_y - b) / profile_X[-1]
    max_a = (min_y - b) / profile_X[-1]
    a = np.random.uniform(min_a, max_a)
    return a * profile_X + b

def exponential_profile_func(profile_X, min_y, max_y):
    b = np.random.uniform(min_y, max_y - 0.0001)
    a = np.random.uniform(50, 700)
    amplitude = np.random.uniform(0.01, 1)
    return (1 - np.exp(-a * profile_X)) * (max_y - b) * amplitude + b

def exponential_profile_func_one_bump(profile_X, min_y, max_y):
    b = np.random.uniform(min_y, max_y)
    a = np.random.uniform(50, 700)
    amplitude = np.random.uniform(0.01, 1)
    bump_amplitude = np.random.uniform(-0.0002, 0.0002)
    bump_mean = np.random.uniform(0.005, 0.015)
    bump_std_dev = np.random.uniform(0.001, 0.005)
    profile = (1 - np.exp(-a * profile_X)) * (max_y - b) * amplitude + b
    bump = bump_amplitude * np.exp(-0.5 * ((profile_X - bump_mean) / bump_std_dev) ** 2)
    return profile + bump

def exponential_profile_func_two_bumps(profile_X, min_y, max_y):
    b = np.random.uniform(min_y, max_y)
    a = np.random.uniform(50, 700)
    amplitude = np.random.uniform(0.01, 1)
    bump_amplitude = np.random.uniform(-0.0002, 0.0002)
    bump_mean = np.random.uniform(0.005, 0.015)
    bump_std_dev = np.random.uniform(0.001, 0.005)
    bump_amplitude2 = np.random.uniform(-0.0002, 0.0002)
    bump_mean2 = np.random.uniform(0.005, 0.015)
    bump_std_dev2 = np.random.uniform(0.001, 0.005)
    profile = (1 - np.exp(-a * profile_X)) * (max_y - b) * amplitude + b
    bump1 = bump_amplitude * np.exp(-0.5 * ((profile_X - bump_mean) / bump_std_dev) ** 2)
    bump2 = bump_amplitude2 * np.exp(-0.5 * ((profile_X - bump_mean2) / bump_std_dev2) ** 2)
    return profile + bump1 + bump2

def degree2_profile_func(profile_X, min_y, max_y):
    b = np.random.uniform(min_y, max_y)
    a1 = np.random.uniform(-10, 10)
    a2 = np.random.uniform(-0.01, 0.01)
    center1 = np.random.uniform(-0.015, 0.035)
    center2 = np.random.uniform(-0.015, 0.035)
    return a1 * (profile_X - center1) ** 2 + a2 * (profile_X - center2) + b

def degree3_profile_func(profile_X, min_y, max_y):
    b = np.random.uniform(min_y, max_y)
    a1 = np.random.uniform(-100, 100)
    a2 = np.random.uniform(-10, 10)
    a3 = np.random.uniform(-0.001, 0.001)
    center1 = np.random.uniform(-0.015, 0.035)
    center2 = np.random.uniform(-0.015, 0.035)
    center3 = np.random.uniform(-0.015, 0.035)
    return a1 * (profile_X - center1) ** 3 + a2 * (profile_X - center2) ** 2 + a3 * (profile_X - center3) + b

def gaussian_profile_func(profile_X, min_y, max_y):
    b = np.random.uniform(min_y, max_y)
    a = np.random.uniform(0.0001, 0.001)
    mean = np.random.uniform(0., 0.02)
    std_dev = np.random.uniform(0.0001, 0.005)
    return a * np.exp(-0.5 * ((profile_X - mean) / std_dev) ** 2) + b

def asymetric_gaussian_profile_func(profile_X, min_y, max_y):
    y = np.zeros_like(profile_X)
    b = np.random.uniform(min_y, max_y)
    a = np.random.uniform(0.0001, 0.001)
    mean = np.random.uniform(0., 0.02)
    std_right = np.random.uniform(0.0001, 0.005)
    std_left = np.random.uniform(0.0001, 0.005)
    left = profile_X < mean
    right = profile_X >= mean
    y[left] = a * np.exp(-0.5 * ((profile_X[left] - mean) / std_left) ** 2) + b
    y[right] = a * np.exp(-0.5 * ((profile_X[right] - mean) / std_right) ** 2) + b
    return y

def mixture_of_gaussians_profile_func(profile_X, min_y, max_y, component_nb):
    b = np.random.uniform(min_y, max_y)
    a = np.random.uniform(0.0001, 0.001)
    means = np.random.uniform(0., 0.02, size=component_nb)
    std_devs = np.random.uniform(0.0001, 0.005, size=component_nb)
    weights = np.random.dirichlet(np.ones(component_nb))
    y = sum(weights[i] * a * np.exp(-0.5 * ((profile_X - means[i]) / std_devs[i]) ** 2) for i in range(component_nb)) + b
    return y

def mixture_of_gaussians_profile_func_2(profile_X, min_y, max_y):
    return mixture_of_gaussians_profile_func(profile_X, min_y, max_y, 2)

def mixture_of_gaussians_profile_func_3(profile_X, min_y, max_y):
    return mixture_of_gaussians_profile_func(profile_X, min_y, max_y, 3)

def mixture_of_gaussians_profile_func_4(profile_X, min_y, max_y):
    return mixture_of_gaussians_profile_func(profile_X, min_y, max_y, 4)

def mixture_of_gaussians_profile_func_5(profile_X, min_y, max_y):
    return mixture_of_gaussians_profile_func(profile_X, min_y, max_y, 5)

def mixture_of_asymetric_gaussians_profile_func(profile_X, min_y, max_y, component_nb):
    b = np.random.uniform(min_y, max_y)
    a = np.random.uniform(0.0001, 0.001)
    means = np.random.uniform(0., 0.02, size=component_nb)
    std_rights = np.random.uniform(0.0001, 0.005, size=component_nb)
    std_lefts = np.random.uniform(0.0001, 0.005, size=component_nb)
    weights = np.random.dirichlet(np.ones(component_nb))
    y = np.zeros_like(profile_X)
    for i in range(component_nb):
        left = profile_X < means[i]
        right = profile_X >= means[i]
        y[left] += weights[i] * a * np.exp(-0.5 * ((profile_X[left] - means[i]) / std_lefts[i]) ** 2)
        y[right] += weights[i] * a * np.exp(-0.5 * ((profile_X[right] - means[i]) / std_rights[i]) ** 2)
    y += b
    return y
    
def mixture_of_asymetric_gaussians_profile_func_2(profile_X, min_y, max_y):
    return mixture_of_asymetric_gaussians_profile_func(profile_X, min_y, max_y, 2)

def mixture_of_asymetric_gaussians_profile_func_3(profile_X, min_y, max_y):
    return mixture_of_asymetric_gaussians_profile_func(profile_X, min_y, max_y, 3)

def mixture_of_asymetric_gaussians_profile_func_4(profile_X, min_y, max_y):
    return mixture_of_asymetric_gaussians_profile_func(profile_X, min_y, max_y, 4)

def mixture_of_asymetric_gaussians_profile_func_5(profile_X, min_y, max_y):
    return mixture_of_asymetric_gaussians_profile_func(profile_X, min_y, max_y, 5)

def sigmoid_with_bump_profile_func(profile_X, min_y, max_y):
    b = np.random.uniform(min_y, max_y - 0.0002)
    L = np.random.uniform(0.0002, 0.001)
    k = np.random.uniform(300, 800)
    x0 = np.random.uniform(0.005, 0.015)
    sigmoid = L / (1 + np.exp(-k * (profile_X - x0))) + b

    # Add a Gaussian bump on top
    bump_amplitude = np.random.uniform(-0.0001, 0.0001)
    bump_mean = np.random.uniform(0.005, 0.015)
    bump_std = np.random.uniform(0.001, 0.003)
    bump = bump_amplitude * np.exp(-0.5 * ((profile_X - bump_mean) / bump_std)**2)

    return sigmoid + bump

def plateau_then_decay_profile_func(profile_X, min_y, max_y):
    plateau_height = np.random.uniform(min_y, max_y)
    decay_start = np.random.uniform(0.005, 0.015)
    decay_rate = np.random.uniform(100, 1000)
    y = np.ones_like(profile_X) * plateau_height
    decay_region = profile_X > decay_start
    y[decay_region] = plateau_height * np.exp(-decay_rate * (profile_X[decay_region] - decay_start))
    return y

def sinusoidal_bump_profile_func(profile_X, min_y, max_y):
    b = np.random.uniform(min_y, max_y - 0.0002)
    amplitude = np.random.uniform(0.0001, 0.0006)
    frequency = np.random.uniform(1, 5)
    phase = np.random.uniform(0, 2*np.pi)
    window_center = np.random.uniform(0.005, 0.015)
    window_width = np.random.uniform(0.005, 0.01)
    window = np.exp(-((profile_X - window_center) / window_width)**2)
    y = b + amplitude * np.sin(2 * np.pi * frequency * profile_X + phase) * window
    return y

def arctan_profile_func(profile_X, min_y, max_y):
    b = np.random.uniform(min_y, max_y - 0.0002)
    a = np.random.uniform(0.0001, 0.001)
    x0 = np.random.uniform(0., 0.02)
    k = np.random.uniform(50, 200)
    return b + a * np.arctan(k * (profile_X - x0))





##############################################
### symmetry decorators for more diversity ###
def z_plane_symmetry(func):
    """
    Decorator to apply z-plane symmetry to a profile function.
    """
    def wrapper(*args, **kwargs):
        profile = func(*args, **kwargs)
        new_profile = np.zeros_like(profile)
        for i in range(len(profile)):
            new_profile[i] = profile[-i-1]
        return new_profile
    return wrapper

def negative_symmetry(func, max_y):
    """
    Decorator to apply negative symmetry to a profile function.
    """
    def wrapper(*args, **kwargs):
        profile = func(*args, **kwargs)
        new_profile = np.zeros_like(profile)
        for i in range(len(profile)):
            new_profile[i] = max_y - profile[-i-1]
        return new_profile
    return wrapper

### -------------------------------------- ###
##############################################

# Parameters
np.random.seed()
nb_per_type = 300
profile_length = 0.02
pts_nb = 40
step = profile_length / pts_nb
profile_X = np.linspace(0, profile_length, pts_nb)
min_y = 0.0002
max_y = 0.0015
derivative_max = 0.1
retry_limit = 500

"""# Generate and plot profiles
profiles = []
fails = 0
for _ in range(nb_per_type):
    profile = generate_profile(inverse_sqrt_profile_func, profile_X, min_y, max_y, derivative_max, step, retry_limit)
    if profile is not None:
        profiles.append(profile)
        plt.plot(profile_X, profile, color='blue', alpha=0.1)
    else:
        fails += 1
print(f"Failed to generate {fails} profiles out of {nb_per_type}")

plt.title('Generated Profiles')
plt.xlabel('Z-axis (m)')
plt.ylabel('Pitch (m)')
plt.grid()
plt.show()"""

all_functions_simple = [
    linear_profile_func,
    exponential_profile_func,
    exponential_profile_func_one_bump,
    exponential_profile_func_two_bumps,
    degree2_profile_func,
    degree3_profile_func,
    gaussian_profile_func,
    asymetric_gaussian_profile_func,
    mixture_of_gaussians_profile_func_2,
    mixture_of_gaussians_profile_func_3,
    mixture_of_gaussians_profile_func_4,
    mixture_of_gaussians_profile_func_5,
    mixture_of_asymetric_gaussians_profile_func_2,
    mixture_of_asymetric_gaussians_profile_func_3,
    mixture_of_asymetric_gaussians_profile_func_4,
    mixture_of_asymetric_gaussians_profile_func_5,
    sigmoid_with_bump_profile_func,
    plateau_then_decay_profile_func,
    sinusoidal_bump_profile_func,
    arctan_profile_func,
]

all_functions_sym = [
    z_plane_symmetry(func) for func in all_functions_simple
]

all_functions_sym_neg = [
    negative_symmetry(func, max_y) for func in all_functions_simple
]

all_functions = all_functions_simple + all_functions_sym + all_functions_sym_neg

profiles = []
fails = 0
for func in all_functions:
    for _ in range(nb_per_type):
        profile = generate_profile(func, profile_X, min_y, max_y, derivative_max, step, retry_limit)
        if profile is not None:
            profiles.append(profile)
        else:
            fails += 1
print(f"Failed to generate {fails} profiles out of {nb_per_type * len(all_functions)}")

save_path = "pitch_profiles/second_database"
import os
if not os.path.exists(save_path):
    os.makedirs(save_path)

key = 999
for i, profile in enumerate(profiles):
    result = np.column_stack((profile_X, profile)).T
    coil_name = f"coil_h_profile_{key}{i}.txt"
    file_path = os.path.join(save_path, coil_name)
    np.savetxt(file_path, result)

Failed to generate 0 profiles out of 180
